# Data Merging Notebook

#### This notebook merges data from the 20_intermediate_files directory and saves a merged dataset for final analysis. We use a notebook for this instead of a script for interpretability across teammates and easier error checking.

#### Imports

In [251]:
import pandas as pd
import numpy as np


#### Import datasets

In [296]:
# set directory
working_dir = "../20_intermediate_files/"

# set file paths
census = working_dir + "census_df.csv"
fips = working_dir + "fips_df.csv"
vital = working_dir + "vital_clean.csv"
wapo = working_dir + "wapo_clean.csv"


In [297]:
# read in data
census_df = pd.read_csv(census)
fips_df = pd.read_csv(fips, dtype={"FIPS": str, "STATE_FIPS": str})
vital_df = pd.read_csv(vital, dtype={"FIPS": str})
wapo_df = pd.read_csv(wapo, dtype={"YEAR": str, "MONTH": str})


#### Merge Prep

***Census Data is low drama***

In [298]:
# take a look at the data starting with census

# rename county_name to match other data
census_df = census_df.rename(columns={"COUNTY_NAME": "county_name"})

# Rename Miami-Dade County to Dade County
census_df.loc[
    census_df["county_name"] == "MIAMI-DADE COUNTY, FL", "county_name"
] = "DADE COUNTY, FL"


In [299]:
# show counties that contain the word DADE
census_df[census_df["county_name"].str.contains("DADE")]


,county_name,POPULATION,STATE
182,"DADE COUNTY, FL",2496435,FL
247,"DADE COUNTY, GA",16633,GA


In [300]:
print(f"There are {census_df.shape[0]} counties in the census data.")


There are 1039 counties in the census data.


In [301]:
# census_df.sample(5)


***Fix the FIPS data next to remove the state-only lines***

In [302]:
# add a new column of the int of the FIPS code
fips_df["FIPS_INT"] = fips_df["FIPS"].astype(int)

# drop the row if the FIPS code is divisible by 100
fips_df = fips_df[fips_df["FIPS_INT"] % 100 != 0]


In [303]:
# show fips for counties that contain the word DADE
# fips_df[fips_df["COUNTY_NAME"].str.contains("DADE")]


In [304]:
# FIPS codes
# fips_df.dtypes
# fips_df.sample(5)


In [305]:
print(f"There are {fips_df.shape[0]} counties in the FIPS data set.")


There are 1039 counties in the FIPS data set.


***Clean the Vital DF to add the new merge key with FIPS code***

In [337]:
# show miami dade county
# vital_df[vital_df["FIPS"] == "12086"]
# vital_df[vital_df["FIPS"] == "12025"]

# because it changed

# search the vital_df for FIPS code 12086 and change it to 12025 and rename the county to DADE COUNTY, FL
vital2 = vital_df.replace("012086", "012025")
vital3 = vital2.replace("MIAMI-DADE COUNTY, FL", "DADE COUNTY, FL")


In [339]:
# show FIPS codes for counties that contain the word DADE
vital3[vital3["COUNTY_NAME"].str.contains("DADE")]


,COUNTY_NAME,FIPS,YEAR,STATE,DRUG,DEATHS
1641,"DADE COUNTY, FL",012025,2003,FL,1,111
1642,"DADE COUNTY, FL",012025,2004,FL,1,145
1643,"DADE COUNTY, FL",012025,2005,FL,1,154
1644,"DADE COUNTY, FL",012025,2006,FL,1,152
1645,"DADE COUNTY, FL",012025,2007,FL,1,149
1646,"DADE COUNTY, FL",012025,2008,FL,1,149
1647,"DADE COUNTY, FL",012025,2009,FL,1,155
1648,"DADE COUNTY, FL",012025,2010,FL,1,112
1649,"DADE COUNTY, FL",012025,2011,FL,1,133
1650,"DADE COUNTY, FL",012025,2012,FL,1,145


***FINALLY***

In [340]:
vital_df = vital3.copy()

# if the FIPS code is six characters, remove the leading zero
vital_df["FIPS"] = vital_df["FIPS"].apply(lambda x: x[1:] if len(x) == 6 else x)

# Add a column for the merge key: County_name+year
vital_df["merge_key"] = vital_df["FIPS"] + vital_df["YEAR"].astype(str)


In [344]:
# find fips of 12025 should be many and 12086 - should be null
vital_df[vital_df["FIPS"] == "12025"]
# vital_df[vital_df["FIPS"] == "12086"]


,COUNTY_NAME,FIPS,YEAR,STATE,DRUG,DEATHS,merge_key
1641,"DADE COUNTY, FL",12025,2003,FL,1,111,120252003
1642,"DADE COUNTY, FL",12025,2004,FL,1,145,120252004
1643,"DADE COUNTY, FL",12025,2005,FL,1,154,120252005
1644,"DADE COUNTY, FL",12025,2006,FL,1,152,120252006
1645,"DADE COUNTY, FL",12025,2007,FL,1,149,120252007
1646,"DADE COUNTY, FL",12025,2008,FL,1,149,120252008
1647,"DADE COUNTY, FL",12025,2009,FL,1,155,120252009
1648,"DADE COUNTY, FL",12025,2010,FL,1,112,120252010
1649,"DADE COUNTY, FL",12025,2011,FL,1,133,120252011
1650,"DADE COUNTY, FL",12025,2012,FL,1,145,120252012


In [346]:
# check for duplicate merge keys
vital_df["merge_key"].duplicated().sum()


0

In [347]:
print(
    f"There are {vital_df.shape[0]} rows in the vital_df after miami-dade corrections."
)


There are 3064 rows in the vital_df after miami-dade corrections.


In [348]:
# show vital df dade where fips is either 12025 or 12086
vital_df[(vital_df["FIPS"] == "12025") | (vital_df["FIPS"] == "12086")]


,COUNTY_NAME,FIPS,YEAR,STATE,DRUG,DEATHS,merge_key
1641,"DADE COUNTY, FL",12025,2003,FL,1,111,120252003
1642,"DADE COUNTY, FL",12025,2004,FL,1,145,120252004
1643,"DADE COUNTY, FL",12025,2005,FL,1,154,120252005
1644,"DADE COUNTY, FL",12025,2006,FL,1,152,120252006
1645,"DADE COUNTY, FL",12025,2007,FL,1,149,120252007
1646,"DADE COUNTY, FL",12025,2008,FL,1,149,120252008
1647,"DADE COUNTY, FL",12025,2009,FL,1,155,120252009
1648,"DADE COUNTY, FL",12025,2010,FL,1,112,120252010
1649,"DADE COUNTY, FL",12025,2011,FL,1,133,120252011
1650,"DADE COUNTY, FL",12025,2012,FL,1,145,120252012


***Prep the WAPO dataset by adding FIPS to county names and generating merge key***

In [349]:
# groupby the COUNTY_NAME, STATE, and YEAR and sum the DRUG_QUANTITY
wapo_df = (
    wapo_df.groupby(["COUNTY_NAME", "STATE", "YEAR"])
    .agg({"QUANTITY": "sum"})
    .reset_index()
)


In [350]:
# wapo_df.sample(10)


In [351]:
# Add FIPS codes to the wapo_df data by merging on county_name
wapo_df_fips = pd.merge(wapo_df, fips_df, on="COUNTY_NAME", how="left").copy()

# Drop STATE_FIPS and STATE_y
wapo_df_fips = wapo_df_fips.drop(["STATE_FIPS", "STATE_y", "FIPS_INT"], axis=1)

# strip the last character from the FIPS column
wapo_df_fips["FIPS"] = wapo_df_fips["FIPS"].str[:-1]

# Add the same merge key
wapo_df_fips["merge_key"] = wapo_df_fips["FIPS"] + wapo_df_fips["YEAR"].astype(str)

# rename the QUANTITY column to DRUG_QUANTITY
wapo_df_fips = wapo_df_fips.rename(
    columns={"QUANTITY": "DRUG_QUANTITY", "STATE_x": "STATE"}
)

# add a flag column to indicate the source of the data
wapo_df_fips["source"] = 1


In [352]:
# WAPO Data
# wapo_df_fips.sample(5)


In [353]:
# make a list of the unique counties in the wapo_df_fips where FIPS is NaN
missing_counties = wapo_df_fips[wapo_df_fips["FIPS"].isna()]["COUNTY_NAME"].unique()


In [354]:
missing_counties


array(['DE BACA COUNTY, NM', 'DE KALB COUNTY, AL', 'DE SOTO COUNTY, FL',
       'DE WITT COUNTY, TX', 'MIAMI-DADE COUNTY, FL',
       'SAINT CLAIR COUNTY, AL', 'SAINT JOHNS COUNTY, FL',
       'SAINT LAWRENCE COUNTY, NY', 'SAINT LUCIE COUNTY, FL'],
      dtype=object)

***A Lot of manual digging took place from this moment forward. Its removed. Couple things identified:***

* De Baca County, NM is DeBaca County, NM
* Saint Lawrence County, NY is St. Lawrence County, NY
* De Kalb County, AL is DeKalb County, AL
* De Soto County, FL is DeSoto County, FL
* De Witt County, TX is Dewitt County, TX
* Dade County, FL is Miami-Dade County, FL
* Saint Clair County, AL is St. Clair County, AL
* Saint Johns County, FL is St. Johns County, FL
* Saint Lucie County, FL is St. Lucie County, FL

In [355]:
# show counties that are in the missing counties list
# wapo_df[wapo_df["COUNTY_NAME"].isin(missing_counties)]


In [356]:
# fix the names
wapo_df["COUNTY_NAME"] = wapo_df["COUNTY_NAME"].replace(
    {
        "DE KALB COUNTY, AL": "DEKALB COUNTY, AL",
        "DE SOTO COUNTY, FL": "DESOTO COUNTY, FL",
        "DE WITT COUNTY, TX": "DEWITT COUNTY, TX",
        "MIAMI-DADE COUNTY, FL": "DADE COUNTY, FL",
        "SAINT CLAIR COUNTY, AL": "ST. CLAIR COUNTY, AL",
        "SAINT JOHNS COUNTY, FL": "ST. JOHNS COUNTY, FL",
        "SAINT LUCIE COUNTY, FL": "ST. LUCIE COUNTY, FL",
        "DE BACA COUNTY, NM": "DEBACA COUNTY, NM",
        "SAINT LAWRENCE COUNTY, NY": "ST. LAWRENCE COUNTY, NY",
    }
)


In [357]:
# show counties that are in the missing counties list
wapo_df[wapo_df["COUNTY_NAME"].isin(missing_counties)]


,COUNTY_NAME,STATE,YEAR,QUANTITY


***Success, let's do the merge over***

In [358]:
# Add FIPS codes to the wapo_df data by merging on county_name
wapo_df_fips = pd.merge(wapo_df, fips_df, on="COUNTY_NAME", how="left").copy()


In [359]:
# Drop STATE_FIPS and STATE_y
wapo_df_fips = wapo_df_fips.drop(["STATE_FIPS", "STATE_y"], axis=1)

# strip the last character from the FIPS column
wapo_df_fips["FIPS"] = wapo_df_fips["FIPS"].str[:-1]

# Add the same merge key
wapo_df_fips["merge_key"] = wapo_df_fips["FIPS"] + wapo_df_fips["YEAR"].astype(str)

# rename the QUANTITY column to DRUG_QUANTITY
wapo_df_fips = wapo_df_fips.rename(
    columns={"QUANTITY": "DRUG_QUANTITY", "STATE_x": "STATE"}
)

# add a flag column to indicate the source of the data
wapo_df_fips["source"] = 1


In [360]:
# show rows with NaN
wapo_df_fips[wapo_df_fips["FIPS"].isna()]


,COUNTY_NAME,STATE,YEAR,DRUG_QUANTITY,FIPS,FIPS_INT,merge_key,source


In [361]:
# wapo_df_fips.sample(10)


In [362]:
print(f"There are {wapo_df_fips.shape[0]} rows in the wapo_df_fips")


There are 6961 rows in the wapo_df_fips


#### Merging

***Make the base DF with records for all counties for all years***

In [363]:
# Make the year list from 2003 to 2015
year_list = [i for i in range(2003, 2016)]

# make a base empty dataframe
base_df = pd.DataFrame()

# Make a loop that populates the dataframe with a row for each county and year
for i in fips_df["FIPS"]:
    for j in year_list:
        # concat the new record to the base_df
        base_df = pd.concat(
            [
                base_df,
                pd.DataFrame(
                    {
                        "county_name": fips_df[fips_df["FIPS"] == i][
                            "COUNTY_NAME"
                        ].values[0],
                        "state": fips_df[fips_df["FIPS"] == i]["STATE"].values[0],
                        "year": j,
                        "fips": i,
                        "merge_key": str(i[:-1]) + str(j),
                    },
                    index=[0],
                ),
            ]
        )

# convert merge key to int
# base_df["merge_key"] = base_df["merge_key"].astype(int)

# reset the index
base_df = base_df.reset_index(drop=True)


In [364]:
# base_df.sample(10)

# show DADE COUNTY, FL
# base_df[base_df["county_name"] == "DADE COUNTY, FL"]


In [366]:
assert len(year_list) * fips_df.shape[0] == base_df.shape[0]


***Left merge to add vital stats***

In [367]:
# Left merge the base_df with the vital_df
base_df_vital = pd.merge(base_df, vital_df, on="merge_key", how="left")


In [368]:
# base_df_vital.head(20)


In [369]:
# count rows where drug is not na
base_df_vital["DRUG"].notna().sum()


3064

In [370]:
base_df_vital.tail(15)


,county_name,state,year,fips,merge_key,COUNTY_NAME,FIPS,YEAR,STATE,DRUG,DEATHS
13492,"WHITMAN COUNTY, WA",WA,2014,53075,530752014,NaN,NaN,NaN,NaN,NaN,NaN
13493,"WHITMAN COUNTY, WA",WA,2015,53075,530752015,NaN,NaN,NaN,NaN,NaN,NaN
13494,"YAKIMA COUNTY, WA",WA,2003,53077,530772003,"YAKIMA COUNTY, WA",53077,2003.0,WA,1.0,10.0
13495,"YAKIMA COUNTY, WA",WA,2004,53077,530772004,"YAKIMA COUNTY, WA",53077,2004.0,WA,1.0,29.0
13496,"YAKIMA COUNTY, WA",WA,2005,53077,530772005,"YAKIMA COUNTY, WA",53077,2005.0,WA,1.0,11.0
13497,"YAKIMA COUNTY, WA",WA,2006,53077,530772006,"YAKIMA COUNTY, WA",53077,2006.0,WA,1.0,12.0
13498,"YAKIMA COUNTY, WA",WA,2007,53077,530772007,"YAKIMA COUNTY, WA",53077,2007.0,WA,1.0,24.0
13499,"YAKIMA COUNTY, WA",WA,2008,53077,530772008,"YAKIMA COUNTY, WA",53077,2008.0,WA,1.0,10.0
13500,"YAKIMA COUNTY, WA",WA,2009,53077,530772009,"YAKIMA COUNTY, WA",53077,2009.0,WA,1.0,17.0
13501,"YAKIMA COUNTY, WA",WA,2010,53077,530772010,"YAKIMA COUNTY, WA",53077,2010.0,WA,1.0,24.0


***We are losing 13 counties from the vital stats merge. We will need to investigate this.***

In [371]:
# show the na county_name
# base_df_vital.tail(15)

# It was miami dade county, fl


In [372]:
assert base_df_vital["DRUG"].sum() == vital_df["DRUG"].sum()


In [373]:
# base_df_vital.tail(15)

# success, drop the extra rows we acquired


In [374]:
# Drop COUNTY_NAME, STATE, FIPS, YEAR, DRUG
base_df_vital = base_df_vital.drop(
    ["COUNTY_NAME", "STATE", "FIPS", "YEAR", "DRUG"], axis=1
)

# rename DEATHS to drug_deaths
base_df_vital = base_df_vital.rename(columns={"DEATHS": "drug_deaths"})


In [375]:
# base_df_vital.sample(10)


***Now merge the wapo df in***

In [376]:
# Left merge the base_df_vital with the wapo_df_fips
base_df_vital_wapo = pd.merge(base_df_vital, wapo_df_fips, on="merge_key", how="outer")


In [377]:
# Looks like we caught all the issues...

# base_df_vital_wapo.tail(10)


In [378]:
# Drop the COUNTY_NAME, STATE, YEAR, FIPS, and FIPS_INT columns
base_df_vital_wapo = base_df_vital_wapo.drop(
    ["COUNTY_NAME", "STATE", "YEAR", "FIPS", "FIPS_INT"], axis=1
)


In [379]:
# all the rows found a home
assert base_df_vital_wapo["source"].sum() == wapo_df_fips["source"].sum()


In [380]:
# drop the source column
base_df_vital_wapo = base_df_vital_wapo.drop("source", axis=1)

# rename the DRUG_QUANTITY column to drug_quantity
base_df_vital_wapo = base_df_vital_wapo.rename(
    columns={"DRUG_QUANTITY": "drug_quantity"}
)


In [381]:
# base_df_vital_wapo.sample(10)


#### Add population

In [382]:
# merge in the population data
base_df_vital_wapo_pop = pd.merge(
    base_df_vital_wapo, census_df, on="county_name", how="left"
)


In [383]:
# show the DADE COUNTY, FL values to verify pop is correct
base_df_vital_wapo_pop[base_df_vital_wapo_pop["county_name"] == "DADE COUNTY, FL"]


,county_name,state,year,fips,merge_key,drug_deaths,drug_quantity,POPULATION,STATE
1976,"DADE COUNTY, FL",FL,2003,12025,120252003,111.0,NaN,2496435.0,FL
1977,"DADE COUNTY, FL",FL,2004,12025,120252004,145.0,NaN,2496435.0,FL
1978,"DADE COUNTY, FL",FL,2005,12025,120252005,154.0,NaN,2496435.0,FL
1979,"DADE COUNTY, FL",FL,2006,12025,120252006,152.0,159171.0,2496435.0,FL
1980,"DADE COUNTY, FL",FL,2007,12025,120252007,149.0,173867.0,2496435.0,FL
1981,"DADE COUNTY, FL",FL,2008,12025,120252008,149.0,205617.0,2496435.0,FL
1982,"DADE COUNTY, FL",FL,2009,12025,120252009,155.0,291639.0,2496435.0,FL
1983,"DADE COUNTY, FL",FL,2010,12025,120252010,112.0,399644.0,2496435.0,FL
1984,"DADE COUNTY, FL",FL,2011,12025,120252011,133.0,397969.0,2496435.0,FL
1985,"DADE COUNTY, FL",FL,2012,12025,120252012,145.0,276012.0,2496435.0,FL


In [384]:
base_df_vital_wapo_pop.tail(15)


,county_name,state,year,fips,merge_key,drug_deaths,drug_quantity,POPULATION,STATE
13492,"WHITMAN COUNTY, WA",WA,2014,53075,530752014,NaN,NaN,44776.0,WA
13493,"WHITMAN COUNTY, WA",WA,2015,53075,530752015,NaN,NaN,44776.0,WA
13494,"YAKIMA COUNTY, WA",WA,2003,53077,530772003,10.0,NaN,243231.0,WA
13495,"YAKIMA COUNTY, WA",WA,2004,53077,530772004,29.0,NaN,243231.0,WA
13496,"YAKIMA COUNTY, WA",WA,2005,53077,530772005,11.0,NaN,243231.0,WA
13497,"YAKIMA COUNTY, WA",WA,2006,53077,530772006,12.0,30448.0,243231.0,WA
13498,"YAKIMA COUNTY, WA",WA,2007,53077,530772007,24.0,33266.0,243231.0,WA
13499,"YAKIMA COUNTY, WA",WA,2008,53077,530772008,10.0,37393.0,243231.0,WA
13500,"YAKIMA COUNTY, WA",WA,2009,53077,530772009,17.0,43570.0,243231.0,WA
13501,"YAKIMA COUNTY, WA",WA,2010,53077,530772010,24.0,51030.0,243231.0,WA


In [385]:
# count the rows where pop is na
# base_df_vital_wapo_pop["POPULATION"].isna().sum()

# drop em
base_df_vital_wapo_pop = base_df_vital_wapo_pop.dropna(subset=["POPULATION"])


In [386]:
assert base_df_vital_wapo_pop.POPULATION.isna().sum() == 0


In [387]:
# show na population rows
base_df_vital_wapo_pop[base_df_vital_wapo_pop["POPULATION"].isna()]


,county_name,state,year,fips,merge_key,drug_deaths,drug_quantity,POPULATION,STATE


In [388]:
# drop the STATE column
base_df_vital_wapo_pop = base_df_vital_wapo_pop.drop("STATE", axis=1)

# rename the POPULATION column to population
base_df_vital_wapo_pop = base_df_vital_wapo_pop.rename(
    columns={"POPULATION": "population"}
)


In [389]:
# base_df_vital_wapo_pop.sample(10)


#### Add columns for per capita rates

In [390]:
grouped_df = base_df_vital_wapo_pop.copy()


In [391]:
# add a column for death rate per 100,000 people
grouped_df["death_rate"] = grouped_df["drug_deaths"] / grouped_df["population"] * 100000

# add a column for ship rate per 100,000 people
grouped_df["ship_rate"] = (
    grouped_df["drug_quantity"] / grouped_df["population"] * 100000
)


In [395]:
# grouped_df.sample(10)


#### Export the final grouped df

In [396]:
# export to the 20_intermediate_files directory
grouped_df.to_csv("../20_intermediate_files/analysis_df.csv", index=False)
